In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all(thread=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

127.0.0.1 - - [2020-03-08 20:22:35] "GET /upload_jobs HTTP/1.1" 200 7015 0.024063
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.002006
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/css/template.css HTTP/1.1" 200 16644 0.002006
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/css/mycss.css HTTP/1.1" 200 2125 0.002005
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88470 0.001971
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21584 0.002011
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60337 0.002011
127.0.0.1 - - [2020-03-08 20:22:35] "GET /static/js/template.js HTTP/1.1" 200 1194 0.002004
127.0.0.1 - - [2020-03-08 20:22:36] "GET /static/vendor/fontawesome-free/webfonts/fa-solid-900.woff2 HTTP/1.1" 200 76394 0.002005
127.0.0.1 - - [2020-03-08 20:22:36] "GET /static/vendor

1111


127.0.0.1 - - [2020-03-08 20:23:49] "GET /run_job/job_world_fond_indexes?report_job_world_fond_indexes=%D0%9E%D0%B1%D0%BD%D0%BE%D0%B2%D0%B8%D1%82%D1%8C%21 HTTP/1.1" 302 547 70.163001
127.0.0.1 - - [2020-03-08 20:23:49] "GET /upload_jobs HTTP/1.1" 200 7390 0.002024


1111


127.0.0.1 - - [2020-03-08 20:27:25] "GET /run_job/job_world_fond_indexes?report_job_world_fond_indexes=%D0%9E%D0%B1%D0%BD%D0%BE%D0%B2%D0%B8%D1%82%D1%8C%21 HTTP/1.1" 302 547 68.210114
127.0.0.1 - - [2020-03-08 20:27:25] "GET /upload_jobs HTTP/1.1" 200 7390 0.003505


In [ ]:
import mygit as mg
from pathlib import Path
import os
folder=os.path.abspath('{}/..'.format(os.getcwd()))
#folder=Path(folder).parent
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
#gt.pyignore()
#git reset --hard origin/master gt.pull()
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
%%time
from sys import path
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

In [ ]:
from datetime import datetime
datetime.now().year

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
a=tradingview()

In [ ]:
if len(history_manager)=={}:
    pass
else:
    'разделяем на новые и старые'
    для старых считаем последние 3 года
    для новых- все
    update таблиц